In [1]:
import numpy as np
import pandas as pd
import altair as alt

In [2]:
#Display all columns
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

#Read CSV
#https://cosaari.github.io/wa7.csv
df = pd.read_csv('Police_Stops.csv', error_bad_lines=False)
#Name index
df.index.name = 'index'
df['index'] = df.index.values
#Fill empty values
df.fillna(0,inplace=True)
#Print data
df.head(10000)

,ObjectId,stopdate,stoptime,streetnbr,streetdir,street,Min_,sex,race,ethnic,Year_of_birth,enfaction,rpmainid,index
index,,,,,,,,,,,,,,
0,1,2018/01/19 00:00:00,30,0,0.0,ARAPAHOE AVE/CANYON BLVD,15,M,W,N,1959,N,3174,0
1,2,2018/01/13 00:00:00,2235,2398,0.0,CANYON BLVD/FOLSOM ST,10,M,W,N,1989,Y,3175,1
2,3,2018/01/19 00:00:00,149,1849,0.0,11TH ST/CANYON | WALNUT ALLEY,5,M,W,N,1996,Y,3176,2
3,4,2018/01/19 00:00:00,149,1849,0.0,11TH ST/CANYON | WALNUT ALLEY,5,M,W,N,1996,Y,3177,3
4,5,2018/01/11 00:00:00,2300,2299,0.0,30TH ST/PEARL ST,5,F,W,N,1998,Y,3178,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8210,8211,2018/12/14 00:00:00,2245,398,0.0,S BROADWAY/DARTMOUTH AVE,10,M,W,N,1981,Y,10930,8210
8211,8212,2018/12/15 00:00:00,1848,1599,0.0,FOLSOM ST/ARAPAHOE AVE,10,F,W,N,1976,Y,10931,8211
8212,8213,2018/12/15 00:00:00,1749,2900,0.0,WALNUT ST,3,M,W,N,1983,N,10932,8212


## Filtering
**Filtering data into traffic stops in the first week of each month to look at the differing initial monthly rate.** 

In [3]:
#Filtering for first week of each month

#for index, row in df.iterrows():
#print(type(row['stopdate']))
newDF = pd.DataFrame()
dayList = list(range(1,8))
monthList = list(range(1,13))

x = df.copy()
x["stopdate"] = pd.to_datetime(df['stopdate'])
for index, row in x.iterrows():
    if (row['stopdate'].month in monthList)  & (row['stopdate'].day in dayList):
        newDF = newDF.append(x.iloc[index])
        newDF = newDF.reindex(x.columns, axis=1)
        

newDF = newDF.sort_values(by = ['stopdate'], ascending = True)

newDF = newDF.reset_index(drop = True)
newDF['index'] = 1

newDF.head(1000)

#for index, row in x.iterrows():
    #print(row['stopdate'].month)

,ObjectId,stopdate,stoptime,streetnbr,streetdir,street,Min_,sex,race,ethnic,Year_of_birth,enfaction,rpmainid,index
0,101.0,2018-01-01,1619.0,4561,0.0,ARAPAHOE AVE/MACARTHUR DR,5.0,M,W,N,1976.0,Y,2862.0,1
1,284.0,2018-01-01,1517.0,699,0.0,30TH ST/BASELINE RD,5.0,M,W,N,1983.0,N,2944.0,1
2,238.0,2018-01-01,1500.0,1799,0.0,BROADWAY/CANYON BLVD,10.0,M,W,N,1980.0,Y,3003.0,1
3,237.0,2018-01-01,1500.0,1799,0.0,BROADWAY/CANYON BLVD,10.0,M,W,N,1982.0,Y,3002.0,1
4,236.0,2018-01-01,1500.0,1799,0.0,BROADWAY/CANYON BLVD,10.0,M,W,N,1992.0,Y,3001.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3621.0,2018-06-07,1245.0,2900,0.0,CANYON BLVD,15.0,M,W,N,1988.0,N,6789.0,1
996,4528.0,2018-07-01,420.0,0,0.0,19TH ST/CANYON BLVD,16.0,M,W,N,1983.0,Y,7342.0,1
997,4572.0,2018-07-01,755.0,1500,0.0,13TH ST,15.0,M,B,N,1998.0,N,7349.0,1
998,4565.0,2018-07-01,800.0,1500,0.0,CANYON BLVD,15.0,M,W,N,1982.0,N,7348.0,1


## Aggregation
**The aggregation consisted of summing the number of minutes in each stop at the specified week
in the specified month, as well as the count for the number of males and number of females.** 

In [4]:
dayList = list(range(1,8))
monthList = list(range(1,13))

month = {}
month['sum1'] = []
month['fCount'] = []
month['mCount'] = []

sum1=sum2=sum3=sum4=sum5=sum6=sum7=sum8=sum9=sum10=sum11=sum12 = 0
fCount1=fCount2=fCount3=fCount4=fCount5=fCount6=fCount7=fCount8=fCount9=fCount10=fCount11=fCount12 = 0
mCount1=mCount2=mCount3=mCount4=mCount5=mCount6=mCount7=mCount8=mCount9=mCount10=mCount11=mCount12 = 0
        
        
for index,row in newDF.iterrows():
    if row['stopdate'].month == 1:
        sum1 += row['Min_']
        if row['sex'] == 'M':
            mCount1 += 1
        if row['sex'] == 'F':
            fCount1 += 1
    if row['stopdate'].month == 2:
        sum2 += row['Min_']
        if row['sex'] == 'M':
            mCount2 += 1
        if row['sex'] == 'F':
            fCount2 += 1
    if row['stopdate'].month == 3:
        sum3 += row['Min_']
        if row['sex'] == 'M':
            mCount3 += 1
        if row['sex'] == 'F':
            fCount3 += 1
    if row['stopdate'].month == 4:
        sum4 += row['Min_']
        if row['sex'] == 'M':
            mCount4 += 1
        if row['sex'] == 'F':
            fCount4 += 1
    if row['stopdate'].month == 5:
        sum5 += row['Min_']
        if row['sex'] == 'M':
            mCount5 += 1
        if row['sex'] == 'F':
            fCount5 += 1
    if row['stopdate'].month == 6:
        sum6 += row['Min_']
        if row['sex'] == 'M':
            mCount6 += 1
        if row['sex'] == 'F':
            fCount6 += 1
    if row['stopdate'].month == 7:
        sum7 += row['Min_']
        if row['sex'] == 'M':
            mCount7 += 1
        if row['sex'] == 'F':
            fCount7 += 1
    if row['stopdate'].month == 8:
        sum8 += row['Min_']
        if row['sex'] == 'M':
            mCount8 += 1
        if row['sex'] == 'F':
            fCount8 += 1
    if row['stopdate'].month == 9:
        sum9 += row['Min_']
        if row['sex'] == 'M':
            mCount9 += 1
        if row['sex'] == 'F':
            fCount9 += 1
    if row['stopdate'].month == 10:
        sum10 += row['Min_']
        if row['sex'] == 'M':
            mCount10 += 1
        if row['sex'] == 'F':
            fCount10 += 1
    if row['stopdate'].month == 11:
        sum11 += row['Min_']
        if row['sex'] == 'M':
            mCount11 += 1
        if row['sex'] == 'F':
            fCount11 += 1
    if row['stopdate'].month == 12:
        sum12 += row['Min_']
        if row['sex'] == 'M':
            mCount12 += 1
        if row['sex'] == 'F':
            fCount12 += 1
    

sumList = [sum1,sum2,sum3,sum4,sum5,sum6,sum7,sum8,sum9,sum10,sum11,sum12]    
fList = [fCount1,fCount2,fCount3,fCount4,fCount5,fCount6,fCount7,fCount8,fCount9,fCount10,fCount11,fCount12] 
mList = [mCount1,mCount2,mCount3,mCount4,mCount5,mCount6,mCount7,mCount8,mCount9,mCount10,mCount11,mCount12] 

for i in sumList:
    month['sum1'].append(i)
for i in fList:
    month['fCount'].append(i)
for i in mList:
    month['mCount'].append(i)
    
    
month

{'sum1': [1185.0,
  1933.0,
  2311.0,
  2597.0,
  1426.0,
  1598.0,
  1945.0,
  3014.0,
  1847.0,
  1428.0,
  1088.0,
  1158.0],
 'fCount': [32, 67, 65, 70, 39, 44, 51, 64, 53, 47, 24, 51],
 'mCount': [77, 114, 136, 138, 82, 121, 110, 131, 117, 82, 73, 57]}

In [5]:
newDF1 = pd.DataFrame(columns = ['index','totMins','totF','totM'])
newDF1['index'] = list(range(1,13))
newDF1['totMins'] = month['sum1']
newDF1['totF'] = month['fCount']
newDF1['totM'] = month['mCount']
newDF1

,index,totMins,totF,totM
0,1,1185.0,32,77
1,2,1933.0,67,114
2,3,2311.0,65,136
3,4,2597.0,70,138
4,5,1426.0,39,82
5,6,1598.0,44,121
6,7,1945.0,51,110
7,8,3014.0,64,131
8,9,1847.0,53,117
9,10,1428.0,47,82


## Visualizations

In [6]:
base = alt.Chart(newDF1).mark_bar(color='red',size=10).encode(
    x=alt.X('index',title='Month'),
    y=alt.Y('totMins',title='Time (min)'),
    color= alt.Color('index', scale=alt.Scale(scheme='goldgreen')),
    tooltip = ["totF","totM"]
).properties(
     title={
    "text": ["Total Number of Minutes Stopped per Month"]},
     width=300,
     height=300
)

base
#base.save('Minutes.html')

alt.Chart(...)

In [7]:
base = alt.Chart(newDF1).mark_bar(color='pink', size= 10).encode(
    x=alt.X('index',title='Month'),
    y=alt.Y('totF',title='Number of Females',scale=alt.Scale(domain=[0, 150])),
    #color= alt.Color('index', scale=alt.Scale(scheme='goldgreen')),
    #tooltip = ["season","episode","year"]
).properties(
     title={
    "text": ["Total Number of Females Stopped"]},
     width=300,
     height=300
)

base1 = alt.Chart(newDF1).mark_bar(color='blue',size=10).encode(
    x=alt.X('index',title='Month'),
    y=alt.Y('totM',title='Number of Males',scale=alt.Scale(domain=[0, 150])),
    #color= alt.Color('index', scale=alt.Scale(scheme='goldgreen')),
    #tooltip = ["season","episode","year"]
).properties(
     title={
    "text": ["Total Number of Males Stopped"]},
     width=300,
     height=300
)

x = base | base1
x
#x.save('FvM.html')

alt.HConcatChart(...)